# Retrieving data from the database

In this tutorial, we show how to retrieve data from a (rather minimal) Korus example database. Jump to the next tutorial (#3), to see how the database was created and populated with data.

## Table of contents
* [Connecting to the database](#connecting-to-the-database)
* [Viewing annotation taxonomies](#viewing-annotation-taxonomies)
* [Retrieving annotations](#retrieving-annotations)
* [Retrieving annotations in Ketos format](#retrieving-annotations-in-ketos-format)
* [Filtering on tags](#filtering-on-tags)
* [Ending the session](#ending-the-session)

## Connecting to the database <a class="anchor" id="connecting-to-the-database"></a>

We begin by importing the necessary modules and opening a connection to the example database,

In [1]:
import sqlite3
import korus.db as kdb

conn = sqlite3.connect("db_t3.sqlite")

ModuleNotFoundError: No module named 'korus'

## Viewing annotation taxonomies <a class="anchor" id="viewing-annotation-taxonomies"></a>

As you may recall from Tutorial 1, the taxonomy used by the acoustic analyst for labelling sound sources and sound types, is saved to the Korus database. The taxonomy can be readily retrieved using the `get_taxonomy` function,

In [2]:
tax = kdb.get_taxonomy(conn) 

The `get_taxonomy` function returns an instance of the `AcousticTaxonomy` class,

In [3]:
type(tax)

korus.tax.AcousticTaxonomy

The `show` method helps us visualize the node structure of the taxonomy,

In [4]:
tax.show(append_name=True)  #sound-source tree

Unknown
├── Anthro [Anthropogenic]
│   └── Boat
│       ├── Engine
│       └── Prop [Propeller]
└── Bio [Biological]
    └── Whale
        ├── HW [Humpback whale]
        └── NARW [North Atlantic right whale]



## Retrieving annotations <a class="anchor" id="retrieving-annotations"></a>

Korus contains the function `filter_annotation` to retrieve annotations based on various search criteria. For example, we can use the `filter_annotation` function to retrieve annotations with a specific (sound-source, sound-type) label,

In [5]:
indices = kdb.filter_annotation(conn, source_type=("NARW","Upcall"))

print(indices)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


Note that the `filter_annotation` function merely returns the indices of the annotations that match our search criteria. We can use the `get_annotations` function to obtain the data associated with these annotations as a Pandas DataFrame,

In [6]:
df = kdb.get_annotations(conn, indices)

print(df)

    job_id  deployment_id  file_id sound_source sound_type  \
0        1              2        2         NARW         LU   
1        1              2        2         NARW         LU   
2        1              2        2         NARW         LU   
3        1              2        2         NARW         LU   
4        1              2        2         NARW         LU   
5        1              2        2         NARW         LU   
6        1              2        2         NARW         LU   
7        1              2        2         NARW         LU   
8        1              2        2         NARW         LU   
9        1              2        2         NARW         LU   
10       1              2        2         NARW         LU   
11       1              2        2         NARW         LU   
12       1              2        2         NARW         LU   
13       1              2        3         NARW         LU   
14       1              2        3         NARW         LU   
15      

Note that although we filtered on the sound type 'Upcall', as defined in 2nd version of the taxonomy, the `filter_annotation` function still helped us find all the annotations that had the "old" sound-type label, 'LU', used in the 1st version of the taxonomy as a label for 'loud' upcalls.

## Retrieving annotations in Ketos format <a class="anchor" id="retrieving-annotations-in-ketos-format"></a>

We could also have specified that we wanted the table formatted to be compatible with format used in [Ketos](https://docs.meridian.cs.dal.ca/ketos/),

In [7]:
df_kt = kdb.get_annotations(conn, indices, format="ketos", label=0)

Let's take a look at the annotation table,

In [9]:
print(df_kt)

                           filename relative_path     start  duration  \
0   audio_20130623T080000.116Z.flac      20130623  1127.340       3.0   
1   audio_20130623T080000.116Z.flac      20130623  1152.026       3.0   
2   audio_20130623T080000.116Z.flac      20130623  1195.278       3.0   
3   audio_20130623T080000.116Z.flac      20130623  1226.142       3.0   
4   audio_20130623T080000.116Z.flac      20130623  1356.681       3.0   
5   audio_20130623T080000.116Z.flac      20130623  1435.982       3.0   
6   audio_20130623T080000.116Z.flac      20130623  1487.788       3.0   
7   audio_20130623T080000.116Z.flac      20130623  1510.170       3.0   
8   audio_20130623T080000.116Z.flac      20130623  1529.095       3.0   
9   audio_20130623T080000.116Z.flac      20130623  1535.080       3.0   
10  audio_20130623T080000.116Z.flac      20130623  1712.872       3.0   
11  audio_20130623T080000.116Z.flac      20130623  1766.751       3.0   
12  audio_20130623T080000.116Z.flac      20130623  

## Filtering on tags <a class="anchor" id="filtering-on-tags"></a>

In addition to filtering on the standardized (sound-source, sound-type) labels, which can also filter on the free-text tags. For example,

In [10]:
indices = kdb.filter_annotation(conn, tag="__AUTO_NEGATIVE__")

print(indices)

[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]


As above, we may use the `get_annotations` function to retrieve the data associated with the annotations.

In [11]:
df_kt = kdb.get_annotations(conn, indices, format="ketos", label=1)

print(df_kt)

{0: '__AUTO_NEGATIVE__'}
                           filename relative_path     start  duration  \
0   audio_20130623T080000.116Z.flac      20130623     0.000  1127.340   
1   audio_20130623T080000.116Z.flac      20130623  1130.340    21.686   
2   audio_20130623T080000.116Z.flac      20130623  1155.026    40.252   
3   audio_20130623T080000.116Z.flac      20130623  1198.278    27.864   
4   audio_20130623T080000.116Z.flac      20130623  1229.142   127.539   
5   audio_20130623T080000.116Z.flac      20130623  1359.681    76.301   
6   audio_20130623T080000.116Z.flac      20130623  1438.982    48.806   
7   audio_20130623T080000.116Z.flac      20130623  1490.788    19.382   
8   audio_20130623T080000.116Z.flac      20130623  1513.170    15.925   
9   audio_20130623T080000.116Z.flac      20130623  1532.095     2.985   
10  audio_20130623T080000.116Z.flac      20130623  1538.080   174.792   
11  audio_20130623T080000.116Z.flac      20130623  1715.872    50.879   
12  audio_20130623T080000.

## Ending the session <a class="anchor" id="ending-the-session"></a>

As the last step of the tutorial, we close the connection to the database.

In [12]:
conn.close()